In [107]:
import pandas as pd
import numpy as np

Import the cdc.csv file from github, I have cleaned some data.

In [108]:
cdc = pd.read_csv("/Users/zainabazizzaveri/Downloads/cdc.csv")

In [278]:
cdc1=cdc.drop(['Primary Mode','Etiology', 'Serotype or Genotype', 'Etiology Status','Setting','Hospitalizations'],axis=1)
cdc1 =cdc1.drop(['Info on Hospitalizations','Deaths','Info on Deaths','Food Contaminated Ingredient','IFSAC Category','Water Exposure','Water Type','Animal Type','Water Status','Animal Type Specify'],axis=1)
cdc1 = cdc1[cdc1.State !='Multistate']
cdc1["sequence item 0: expected str instance, float found"]

,Year,Month,State,Illnesses,Food Vehicle
0,2017,1,NY,20,"house salad; fish, baked flounder; broccoli, u..."
1,2017,1,WI,7,NaN
2,2017,1,TN,10,NaN
3,2017,1,OH,14,NaN
4,2017,1,WI,35,NaN
...,...,...,...,...,...
20843,1998,1,OH,4,"pizza, pepperoni"
20844,1998,1,CA,4,"stuffing, unspecified; sandwich, turkey"
20845,1998,1,PA,17,NaN
20846,1998,8,HI,38,"eggs, other"


In [ ]:
cdc1 = cdc1.drop_duplicates(['Year','Month','State','Illnesses','Food Vehicle'],keep= 'last')
cdc1['str_year'] = cdc1['Year'].astype(str)
cdc1['str_month'] = cdc1['Month'].astype(str)
cdc1['conc'] = cdc1['str_year'] + cdc1['str_month'] + cdc1['State']
cdc1.loc[cdc1['conc']== '20164AZ']
cdc1['Illnesses'].astype(str)
cdc1.info()

In [309]:
cdc1.loc[cdc1['conc']== '199810CA']

,Year,Month,State,Illnesses,Food Vehicle,str_year,str_month,conc
20206,1998,10,CA,64,NaN,1998,10,199810CA
20333,1998,10,CA,19,"chile relleno, unspecified",1998,10,199810CA
20338,1998,10,CA,4,"eggs, unspecified",1998,10,199810CA
20350,1998,10,CA,9,NaN,1998,10,199810CA
20359,1998,10,CA,15,turkey salad,1998,10,199810CA
20361,1998,10,CA,3,"tuna, unspecified",1998,10,199810CA
20363,1998,10,CA,39,NaN,1998,10,199810CA
20374,1998,10,CA,209,NaN,1998,10,199810CA


In [ ]:
cdc2 = cdc1['Food Vehicle'].groupby([cdc1.conc]).apply(list).reset_index()
cdc3 = cdc1['Illnesses'].groupby([cdc1.conc]).apply(list).reset_index()

In [318]:
cdc4 = cdc2.merge(cdc3, left_on='conc', right_on='conc')
cdc4

,conc,Food Vehicle,Illnesses
0,199810CA,"[nan, chile relleno, unspecified, eggs, unspec...","[64, 19, 4, 9, 15, 3, 39, 209]"
1,199810CO,"[hollandaise sauce; fish, halibut; crab, unspe...",[14]
2,199810FL,"[gumbo, unspecified, chicken, unspecified, tor...","[5, 4, 30, 2, 2, 2, 4, 3, 2, 3, 7, 3, 2, 14, 2..."
3,199810HI,"[fish, mahi mahi, fish eggs, unspecified, fish...","[2, 6, 2, 5, 48]"
4,199810IA,"[soup, pork-based; pork, other; corn, unspecif...",[17]
...,...,...,...
6410,20179UT,"[fish, mahi mahi]",[2]
6411,20179VA,[clam chowder],[171]
6412,20179VT,"[ground beef, hamburger]",[2]
6413,20179WA,"[oysters, raw, oysters, raw, pinto beans, whol...","[6, 4, 2, 4]"


Give path to the table generated by the DBSCAN noteboook

In [243]:
tab1 = pd.read_csv("/Users/zainabazizzaveri/Downloads/tables/table0.05-3-9.csv")

In [281]:
y = tab1.rename(columns={"Unnamed: 0": "Cluster_Number", "cluster": "size_of_clust", "date": "start_date", "date.1":"end_date"}, errors="raise")

In [282]:
x= y.drop([0,1])
date = pd.to_datetime(x['start_date'])
x['start_date'] = date
date = pd.to_datetime(x['end_date'])
x['end_date'] = date
x["s_month"] = x['start_date'].map(lambda x: x.month)
x['str_s_month'] = x['s_month'].astype(str)
x["s_year"] = x['start_date'].map(lambda x: x.year)
x['str_s_year'] = x['s_year'].astype(str)
x["e_month"] = x['end_date'].map(lambda x: x.month)
x['str_e_month'] = x['e_month'].astype(str)
x["e_year"] = x['end_date'].map(lambda x: x.year)
x['str_e_year'] = x['e_year'].astype(str)
x =x.drop(['start_date','end_date'],axis=1)

In [283]:
x['diff'] = x['e_year']-x['s_year']
x['valid'] = x['diff'].apply(lambda y: 1 if y<=1 else 0)
x = x[x.valid != 0]

In [284]:
x['conc1'] = x['str_s_year'] + x['str_s_month'] + x['state']
x['conc2'] = x['str_s_year'] + x['str_s_month'] + x['state.1']
x['conc3'] = x['str_e_year'] + x['str_e_month'] + x['state']
x['conc4'] = x['str_e_year'] + x['str_e_month'] + x['state.1']

In [340]:
x['r1'] = x['conc1'].isin(cdc1['conc'])
x['r2'] = x['conc2'].isin(cdc1['conc'])
x['r3'] = x['conc3'].isin(cdc1['conc'])
x['r4'] = x['conc4'].isin(cdc1['conc'])
x['match'] =np.nan
x['r1'] = x['r1'].apply(lambda y: 1 if y == True else 0)
x['r2'] = x['r2'].apply(lambda y: 1 if y == True else 0)
x['r3'] = x['r3'].apply(lambda y: 1 if y == True else 0)
x['r4'] = x['r4'].apply(lambda y: 1 if y == True else 0)
x['sumr']   =    x['r1'] + x['r2'] +x['r3'] +x['r4'] 
x['match']= x['sumr'].apply(lambda y: 1 if y>0 else 0)
#x.rename({'conc1': 'conc'}, axis=1)

In [360]:
#y = x.join(cdc4, on='conc', how = 'outer')
#y = pd.merge(x, cdc4, on='conc', how='left')
y = x.merge(cdc4, left_on='conc1', right_on='conc',how='left')

In [361]:
#y.rename({'conc':'cat1','Food Vehicle':'fv1','Illnesses':'i1'},axis=1 )

In [362]:
z = y.merge(cdc4, left_on='conc2', right_on='conc',how='left')

In [371]:
p = z.merge(cdc4, left_on='conc3', right_on='conc',how='left')
q = p.merge(cdc4, left_on='conc4', right_on='conc',how='left')
q.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131 entries, 0 to 130
Data columns (total 36 columns):
Cluster_Number    131 non-null object
size_of_clust     131 non-null object
state             131 non-null object
state.1           131 non-null object
s_month           131 non-null int64
str_s_month       131 non-null object
s_year            131 non-null int64
str_s_year        131 non-null object
e_month           131 non-null int64
str_e_month       131 non-null object
e_year            131 non-null int64
str_e_year        131 non-null object
diff              131 non-null int64
valid             131 non-null int64
conc1             131 non-null object
conc2             131 non-null object
conc3             131 non-null object
conc4             131 non-null object
r1                131 non-null int64
r2                131 non-null int64
r3                131 non-null int64
r4                131 non-null int64
match             131 non-null int64
sumr              131 non-null i

This is the total number of matches 

In [372]:
q= q.drop(columns=['str_s_month','str_s_year','str_e_month','str_e_year','diff','valid','conc1','conc2','conc3','conc4','r1','r2','r3','r4','sumr'],axis=1)

In [376]:
q= q.drop(columns=['conc_x','conc_y'],axis=1)

In [379]:
q.head()

,Cluster_Number,size_of_clust,state,state.1,s_month,s_year,e_month,e_year,match,Food Vehicle_x,Illnesses_x,Food Vehicle_y,Illnesses_y,Food Vehicle_x,Illnesses_x,Food Vehicle_y,Illnesses_y
0,0,114,AZ,AZ,4,2016,6,2016,1,"[nan, nan, nan]","[11, 15, 14]","[nan, nan, nan]","[11, 15, 14]","[shrimp, unfried]",[50],"[shrimp, unfried]",[50]
1,1,31,AZ,AZ,5,2014,6,2014,1,[nan],[23],[nan],[23],"[oysters, raw, nan]","[2, 2]","[oysters, raw, nan]","[2, 2]"
2,2,69,AZ,AZ,8,2016,9,2016,1,NaN,NaN,NaN,NaN,[nan],[4],[nan],[4]
3,3,693,NV,NV,2,2017,9,2017,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,19,AZ,AZ,1,2016,1,2016,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [378]:
q['match'].sum()

79